# Importação das bibliotecas necessárias

In [1]:
import pandas as pd 

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from scipy.stats import uniform

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn import set_config
set_config(display='diagram')
import datetime
import time

# Importação dos dados

In [2]:
data = pd.read_parquet('../dataset/processed/artigos_tratados/bertimbau/artigos_tratados_bert_lg.parquet')

In [3]:
# remocao de dados nulos
data = data[data['Conteudo'] != '']

In [4]:
# remocao de colunas desnecessarias
rem_cols = ['Conteudo', 'URL']
data.drop(rem_cols, axis=1, inplace=True)

In [5]:
data.head() # visualização das primeiras 5 linhas do dataframe

,Partido,Vies,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1015,emb_1016,emb_1017,emb_1018,emb_1019,emb_1020,emb_1021,emb_1022,emb_1023,emb_1024
0,Novo,direita,0.401686,-0.142705,0.023679,0.192193,0.323548,-0.067125,-0.598261,-0.355111,...,0.186607,-0.147967,-0.487217,0.110254,-0.125077,-0.159934,0.125114,0.134895,-0.527953,0.196096
1,Novo,direita,-0.128541,0.215314,-0.069348,0.088915,0.408865,-0.052515,-0.267552,-0.157830,...,0.552647,0.091742,-0.605148,0.112920,0.099640,-0.174978,0.192274,0.250751,-0.157322,0.310022
2,Novo,direita,0.016604,0.137099,-0.113500,0.109841,0.290769,0.015612,-0.275260,-0.178999,...,0.497988,0.113761,-0.157607,0.001557,0.053836,0.133553,-0.021814,0.095863,-0.137130,0.318082
3,Novo,direita,-0.032087,-0.067949,-0.036270,0.208884,-0.090851,0.005983,-0.093461,-0.463273,...,0.601618,0.132886,-0.403809,0.213245,-0.007711,-0.157867,-0.053459,0.401732,-0.279196,0.165060
4,Novo,direita,0.181898,0.011968,-0.062858,0.162305,0.247084,0.055331,-0.518530,-0.171481,...,0.447449,-0.020970,-0.419976,0.198067,-0.142836,-0.334448,-0.083704,0.290215,-0.176763,0.452248


In [6]:
# conversao dos rotulos categoricos para numericos
data['Vies'] = data['Vies'].map({'direita':2,
                                'centro': 1,
                                'esquerda': 0})

In [7]:
data.head()

,Partido,Vies,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_1015,emb_1016,emb_1017,emb_1018,emb_1019,emb_1020,emb_1021,emb_1022,emb_1023,emb_1024
0,Novo,2,0.401686,-0.142705,0.023679,0.192193,0.323548,-0.067125,-0.598261,-0.355111,...,0.186607,-0.147967,-0.487217,0.110254,-0.125077,-0.159934,0.125114,0.134895,-0.527953,0.196096
1,Novo,2,-0.128541,0.215314,-0.069348,0.088915,0.408865,-0.052515,-0.267552,-0.157830,...,0.552647,0.091742,-0.605148,0.112920,0.099640,-0.174978,0.192274,0.250751,-0.157322,0.310022
2,Novo,2,0.016604,0.137099,-0.113500,0.109841,0.290769,0.015612,-0.275260,-0.178999,...,0.497988,0.113761,-0.157607,0.001557,0.053836,0.133553,-0.021814,0.095863,-0.137130,0.318082
3,Novo,2,-0.032087,-0.067949,-0.036270,0.208884,-0.090851,0.005983,-0.093461,-0.463273,...,0.601618,0.132886,-0.403809,0.213245,-0.007711,-0.157867,-0.053459,0.401732,-0.279196,0.165060
4,Novo,2,0.181898,0.011968,-0.062858,0.162305,0.247084,0.055331,-0.518530,-0.171481,...,0.447449,-0.020970,-0.419976,0.198067,-0.142836,-0.334448,-0.083704,0.290215,-0.176763,0.452248


# Splits que serão avaliados

In [8]:
# a seguir os dados serão divididos entre features (X) e label (y)

X_columns = [column for column in data.columns if column != 'Vies']
X = data[X_columns] # features
X.head() 

,Partido,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_1015,emb_1016,emb_1017,emb_1018,emb_1019,emb_1020,emb_1021,emb_1022,emb_1023,emb_1024
0,Novo,0.401686,-0.142705,0.023679,0.192193,0.323548,-0.067125,-0.598261,-0.355111,0.149750,...,0.186607,-0.147967,-0.487217,0.110254,-0.125077,-0.159934,0.125114,0.134895,-0.527953,0.196096
1,Novo,-0.128541,0.215314,-0.069348,0.088915,0.408865,-0.052515,-0.267552,-0.157830,-0.072159,...,0.552647,0.091742,-0.605148,0.112920,0.099640,-0.174978,0.192274,0.250751,-0.157322,0.310022
2,Novo,0.016604,0.137099,-0.113500,0.109841,0.290769,0.015612,-0.275260,-0.178999,-0.177130,...,0.497988,0.113761,-0.157607,0.001557,0.053836,0.133553,-0.021814,0.095863,-0.137130,0.318082
3,Novo,-0.032087,-0.067949,-0.036270,0.208884,-0.090851,0.005983,-0.093461,-0.463273,-0.088330,...,0.601618,0.132886,-0.403809,0.213245,-0.007711,-0.157867,-0.053459,0.401732,-0.279196,0.165060
4,Novo,0.181898,0.011968,-0.062858,0.162305,0.247084,0.055331,-0.518530,-0.171481,0.040109,...,0.447449,-0.020970,-0.419976,0.198067,-0.142836,-0.334448,-0.083704,0.290215,-0.176763,0.452248


In [9]:
y = data['Vies'] # label
y.head()

0    2
1    2
2    2
3    2
4    2
Name: Vies, dtype: int64

## Estratificação por viés

In [10]:
X_train_strat_vies, X_test_strat_vies, y_train_strat_vies, y_test_strat_vies = train_test_split(X, y,
                                                                                                test_size=0.2,
                                                                                                random_state=42,
                                                                                                stratify=y)

X_train_strat_vies.drop('Partido', axis=1, inplace=True) # remocao da coluna partido
X_test_strat_vies.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

## Estratificação por partido

In [11]:
X_train_strat_part, X_test_strat_part, y_train_strat_part, y_test_strat_part = train_test_split(X, y,
                                                                                                test_size=0.2,
                                                                                                random_state=42,
                                                                                                stratify=data['Partido'])

X_train_strat_part.drop('Partido', axis=1, inplace=True) # remocao da coluna partido
X_test_strat_part.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

## Teste com partidos fora do treino

In [12]:
direita = data[data['Vies'] == 2] # selecao apenas dos partidos de direita
centro = data[data['Vies'] == 1] # selecao apenas dos partidos de centro
esquerda = data[data['Vies'] == 0] # selecao apenas dos partidos de esquerda
total = data.shape[0] # quantidade total de linhas no dataset

In [13]:
print('Porcentagem dos partidos de direita em relação ao dataset:')
for part in direita['Partido'].unique():
    
    qnt_part = direita[direita['Partido'] == part].shape[0]
    porc = qnt_part / total * 100
    print(f'porcentagem do partido {part}: {porc:.2f}%')

Porcentagem dos partidos de direita em relação ao dataset:
porcentagem do partido Novo: 10.50%
porcentagem do partido PL: 28.62%
porcentagem do partido PP: 5.21%
porcentagem do partido União Brasil: 2.42%


In [14]:
print('Porcentagem dos partidos de centro em relação ao dataset:')
for part in centro['Partido'].unique():
    
    qnt_part = centro[centro['Partido'] == part].shape[0]
    porc = qnt_part / total * 100
    print(f'porcentagem do partido {part}: {porc:.2f}%')

Porcentagem dos partidos de centro em relação ao dataset:
porcentagem do partido PDT: 3.99%
porcentagem do partido MDB: 4.69%
porcentagem do partido PSB: 15.79%
porcentagem do partido PV: 7.36%


In [15]:
print('Porcentagem dos partidos de esquerda em relação ao dataset:')
for part in esquerda['Partido'].unique():
    
    qnt_part = esquerda[esquerda['Partido'] == part].shape[0]
    porc = qnt_part / total * 100
    print(f'porcentagem do partido {part}: {porc:.2f}%')

Porcentagem dos partidos de esquerda em relação ao dataset:
porcentagem do partido PCB: 4.98%
porcentagem do partido PSOL: 0.14%
porcentagem do partido PSTU: 5.62%
porcentagem do partido PCDoB: 5.44%
porcentagem do partido PT: 4.43%
porcentagem do partido Rede: 0.80%


Baseado nas porcentagens acima, o partido de esquerda, centro e direita que foram escolhidos para o conjunto de teste representam, respectivamente 5.53%, 7.35% e 10.34% do dataset. Dessa forma, o conjunto de teste será constituído por 5.53 + 7.35 + 10.34 = 23.22% do dataset original.

In [16]:
part_teste = ['PSTU', 'PV', 'Novo'] # partidos do conjunto de teste

test = data[data['Partido'].isin(part_teste)].copy() # selecao dos dados de teste
test.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

train = data[~data['Partido'].isin(part_teste)].copy() # selecao dos dados de treino
train.drop('Partido', axis=1, inplace=True) # remocao da coluna partido

In [17]:
X_train_part_novos = train.drop('Vies', axis=1) # X_train
y_train_part_novos = train['Vies'] # y_train

X_test_part_novos = test.drop('Vies', axis=1) # X_test
y_test_part_novos = test['Vies'] # y_test

# Seleção do modelo

In [18]:
def seleciona_grid_bert(model):

    param_grid = None

    if isinstance(model, MultinomialNB):
            param_grid = {
            "selection__k": [200,400,600,800,1024],
            "estimator__alpha": [50, 15, 10, 5, 1, 0.5, 0.3, 0.1, 0.05, 0.03, 0.02, 0.01,  0.001],
            "estimator__fit_prior": [True, False],
            }

    if isinstance(model, SVC):
            param_grid = {
            "selection__k": [200,400,600,800,1024],
            "estimator__gamma": [1, 0.1, 0.01, 0.001],
            "estimator__kernel": ['linear', 'sigmoid'],
            "estimator__C": [0.1, 1, 10, 100]
            }
            
    if isinstance(model, LinearSVC):
            param_grid = {
            "selection__k": [200,400,600,800,1024],
            "estimator__dual": [True, False],
            "estimator__penalty": ['l1', 'l2'],
            "estimator__fit_intercept": [True, False],
            "estimator__C": uniform(loc=0, scale=4)
            }


    if isinstance(model, RandomForestClassifier):
        param_grid = {
        "selection__k": [200,400,600,800,1024],
        "estimator__n_estimators": np.arange(20,150), 
        "estimator__max_features": ['log2', 'sqrt'],
        "estimator__max_depth": np.arange(10,110),
        "estimator__min_samples_split": np.arange(2,11),
        "estimator__min_samples_leaf": np.arange(1,5),
        "estimator__bootstrap": [True, False]
        }
        
    if isinstance(model, XGBClassifier):
        param_grid = {
        "selection__k": [200,400,600,800,1024],
        "estimator__gamma": np.linspace(0,9,100, dtype=np.int64),
        "estimator__alpha": np.linspace(0,40,100, dtype=np.int64),
        "estimator__lambda": np.linspace(0,3,10, dtype=np.int64),
        "estimator__colsample_bytree": np.linspace(0.2,1,10, dtype=np.int64)
        }

    return param_grid

In [19]:
def fit_e_avalia(split, random_search):
    
    inicio_random_search = datetime.datetime.now()
    
    if split == 'strat_vies': # estratificacao pela label
        model_trained = random_search.fit(X_train_strat_vies, y_train_strat_vies) # fit

        fim_random_search = datetime.datetime.now()
        tempo_total = fim_random_search - inicio_random_search
        print(f'Duração da Random Search: {tempo_total}')

        y_pred = model_trained.predict(X_test_strat_vies) # predicao
        f1 = f1_score(y_test_strat_vies, y_pred, average= 'macro') # f1
        report = classification_report(y_test_strat_vies, y_pred, output_dict=True) # class report

    elif split == 'strat_partido': # estratificacao pelos partidos 
        model_trained = random_search.fit(X_train_strat_part, y_train_strat_part) # fit

        fim_random_search = datetime.datetime.now()
        tempo_total = fim_random_search - inicio_random_search
        print(f'Duração da Random Search: {tempo_total}')

        y_pred = model_trained.predict(X_test_strat_part) # predicao
        f1 = f1_score(y_test_strat_part, y_pred, average= 'macro') # f1
        report = classification_report(y_test_strat_part, y_pred, output_dict=True) # class report

    elif split == 'pred_partido_novo': # predicao de partidos nao vistos no teste
        model_trained = random_search.fit(X_train_part_novos, y_train_part_novos) # fit

        fim_random_search = datetime.datetime.now()
        tempo_total = fim_random_search - inicio_random_search
        print(f'Duração da Random Search: {tempo_total}')

        y_pred = model_trained.predict(X_test_part_novos) # predicao
        f1 = f1_score(y_test_part_novos, y_pred, average= 'macro') # f1
        report = classification_report(y_test_part_novos, y_pred, output_dict=True) # class report
    
    return model_trained, tempo_total, f1, report

In [20]:
def compara_bert(iteracoes, modelos, nome_arquivo):

    # seletor de features
    selection = SelectKBest() 

    # possibilidades de oversampling ou nao
    samplers = [RandomOverSampler(random_state=42), None]

    # diferentes splits que serao avaliador
    splits = ['strat_vies', 'strat_partido', 'pred_partido_novo'] 

    # dataframe em que sera inserido os dados do modelo testado 
    df_resultados = pd.DataFrame(columns=['modelo', 'split', 'sampler', 'scaling',
                                          'duracao_random_search','qnt_iteracoes',
                                          'f1_randsearch',
                                          'melhores_parametros', 'f1_pred',
                                          'class_report'])
    
    for model in modelos:

        for sampler in samplers:

            for split in splits:
                
                # seleciona grid de parametros
                param_grid = seleciona_grid_bert(model)
                
                
                scaler = MaxAbsScaler()
        
                # define o pipeline
                pipeline = Pipeline([
                        ('scaling', scaler), 
                        ('selection', selection),
                        ('ros', sampler),
                        ('estimator', model)
                        ])
        
                
                #  --- Prints das configurações dessa iteracao ---
                print(f'Modelo: {model}')
                print(f'Split: {split}')
                print(f'Scaler: {scaler}')
                print(f'Sampler: {sampler}')
                    
        
                # definicao da randomized search
                random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,cv=StratifiedKFold(n_splits=5),
                                                    n_iter=iteracoes, n_jobs=1, random_state=42, scoring='f1_macro')


                # fit e avaliacao pela randomized search
                model_trained, tempo_total, f1, report = fit_e_avalia(split, random_search)
                    
                print('---')
                resultados = model_trained.cv_results_

                for params, score in zip(resultados['params'], resultados['mean_test_score']):
                    print(f"Parâmetros: {params}, Score: {score}")
                print('---')    
                    
                # melhor metrica na random search
                score_random_search = model_trained.best_score_
                score_random_search *= 100
                score_random_search = round(score_random_search,2)
                print(f'Melhor F1 na Random Search: {score_random_search}%')
                
                # melhores parametros encontrados
                print('Melhores parâmetros encontrados:')
                print(model_trained.best_params_)

                
                # acuracia da predicao
                f1 *= 100
                f1 = round(f1,2)
                print(f'F1 macro = {f1}%')
        
                # classification report
                print(report)
                        
                
                print('----------------------------------------------')
                
                # --- Escrita em memória secundária ---

                # Nova linha que sera adicionada
                nova_linha = {'modelo': model, 'split': split,
                              'sampler': str(sampler), 'scaling': scaler,
                              'duracao_random_search': tempo_total,
                              'qnt_iteracoes': iteracoes,
                              'f1_randsearch': f'{score_random_search}%',
                              'melhores_parametros': str(model_trained.best_params_),
                              'f1_pred': f'{f1}%', 'class_report': report}
            
                # Cria um novo DataFrame com a nova linha
                nova_linha_resultados = pd.DataFrame([nova_linha])
            
                # Concatena o novo DataFrame com o DataFrame existente
                df_resultados = pd.concat([df_resultados, nova_linha_resultados], ignore_index=True)

                # salvamento do dataframe de resultados apos os testes terem terminado
                df_resultados.to_csv(nome_arquivo, index=False)
    
    
    print('Fim dos testes')
    
    # salvamento do dataframe de resultados apos os testes terem terminado
    #df_resultados.to_csv(nome_arquivo, index=False)

In [21]:
modelos = [XGBClassifier(seed=42, tree_method='gpu_hist', gpu_id=0)]

compara_bert(100, modelos, 'compara-xgb-bert-100-.csv')

Modelo: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)
Split: strat_vies
Scaler: MaxAbsScaler()
Sampler: RandomOverSampler(random_state=42)


D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\xgboost\data.py:427: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

Duração da Random Search: 0:30:38.921428


D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


---
Parâmetros: {'selection__k': 800, 'estimator__lambda': 2, 'estimator__gamma': 4, 'estimator__colsample_bytree': 0, 'estimator__alpha': 13}, Score: 0.8444535970862574
Parâmetros: {'selection__k': 600, 'estimator__lambda': 1, 'estimator__gamma': 8, 'estimator__colsample_bytree': 0, 'estimator__alpha': 34}, Score: 0.8347010252065423
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 2, 'estimator__gamma': 2, 'estimator__colsample_bytree': 0, 'estimator__alpha': 39}, Score: 0.8351368813748173
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 2, 'estimator__gamma': 8, 'estimator__colsample_bytree': 0, 'estimator__alpha': 17}, Score: 0.8356369529633975
Parâmetros: {'selection__k': 400, 'estimator__lambda': 0, 'estimator__gamma': 0, 'estimator__colsample_bytree': 0, 'estimator__alpha': 12}, Score: 0.8409870240254472
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 1, 'estimator__gamma': 1, 'estimator__colsample_bytree': 0, 'estimator__alpha': 8}, Score: 0.846018555022602

...

D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\xgboost\data.py:427: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseD

Duração da Random Search: 0:24:12.951595
---
Parâmetros: {'selection__k': 800, 'estimator__lambda': 2, 'estimator__gamma': 4, 'estimator__colsample_bytree': 0, 'estimator__alpha': 13}, Score: 0.831465855315988
Parâmetros: {'selection__k': 600, 'estimator__lambda': 1, 'estimator__gamma': 8, 'estimator__colsample_bytree': 0, 'estimator__alpha': 34}, Score: 0.8260720459272693
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 2, 'estimator__gamma': 2, 'estimator__colsample_bytree': 0, 'estimator__alpha': 39}, Score: 0.8235047694455684
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 2, 'estimator__gamma': 8, 'estimator__colsample_bytree': 0, 'estimator__alpha': 17}, Score: 0.8298271036948884
Parâmetros: {'selection__k': 400, 'estimator__lambda': 0, 'estimator__gamma': 0, 'estimator__colsample_bytree': 0, 'estimator__alpha': 12}, Score: 0.845023228568766
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 1, 'estimator__gamma': 1, 'estimator__colsample_bytree': 0, 'estimato

D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\xgboost\data.py:427: FutureWarning: is_sparse is deprecated and will be removed in 

Duração da Random Search: 0:22:10.833898
---
Parâmetros: {'selection__k': 800, 'estimator__lambda': 2, 'estimator__gamma': 4, 'estimator__colsample_bytree': 0, 'estimator__alpha': 13}, Score: 0.7285013140019663
Parâmetros: {'selection__k': 600, 'estimator__lambda': 1, 'estimator__gamma': 8, 'estimator__colsample_bytree': 0, 'estimator__alpha': 34}, Score: 0.7181410972411084
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 2, 'estimator__gamma': 2, 'estimator__colsample_bytree': 0, 'estimator__alpha': 39}, Score: 0.7180308909345158
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 2, 'estimator__gamma': 8, 'estimator__colsample_bytree': 0, 'estimator__alpha': 17}, Score: 0.7297076410637602
Parâmetros: {'selection__k': 400, 'estimator__lambda': 0, 'estimator__gamma': 0, 'estimator__colsample_bytree': 0, 'estimator__alpha': 12}, Score: 0.7508383421715816
Parâmetros: {'selection__k': 1024, 'estimator__lambda': 1, 'estimator__gamma': 1, 'estimator__colsample_bytree': 0, 'estima

D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [23]:
resultados = pd.read_csv('compara-xgb-bert-100-.csv')
resultados

,modelo,split,sampler,scaling,duracao_random_search,qnt_iteracoes,f1_randsearch,melhores_parametros,f1_pred,class_report
0,"XGBClassifier(base_score=None, booster=None, c...",strat_vies,RandomOverSampler(random_state=42),MaxAbsScaler(),0 days 00:30:38.921428,100,90.71%,"{'selection__k': 800, 'estimator__lambda': 0, ...",91.48%,"{'0': {'precision': 0.9182242990654206, 'recal..."
1,"XGBClassifier(base_score=None, booster=None, c...",strat_partido,RandomOverSampler(random_state=42),MaxAbsScaler(),0 days 00:28:56.746834,100,90.86%,"{'selection__k': 800, 'estimator__lambda': 0, ...",90.9%,"{'0': {'precision': 0.8831460674157303, 'recal..."
2,"XGBClassifier(base_score=None, booster=None, c...",pred_partido_novo,RandomOverSampler(random_state=42),MaxAbsScaler(),0 days 00:28:21.603563,100,80.01%,"{'selection__k': 800, 'estimator__lambda': 0, ...",81.73%,"{'0': {'precision': 0.7453183520599251, 'recal..."
3,"XGBClassifier(base_score=None, booster=None, c...",strat_vies,NaN,MaxAbsScaler(),0 days 00:24:41.510090,100,89.98%,"{'selection__k': 1024, 'estimator__lambda': 1,...",91.48%,"{'0': {'precision': 0.9347826086956522, 'recal..."
4,"XGBClassifier(base_score=None, booster=None, c...",strat_partido,NaN,MaxAbsScaler(),0 days 00:24:12.951595,100,90.54%,"{'selection__k': 800, 'estimator__lambda': 0, ...",90.87%,"{'0': {'precision': 0.9154589371980676, 'recal..."
5,"XGBClassifier(base_score=None, booster=None, c...",pred_partido_novo,NaN,MaxAbsScaler(),0 days 00:22:10.833898,100,79.78%,"{'selection__k': 800, 'estimator__lambda': 0, ...",80.99%,"{'0': {'precision': 0.7777777777777778, 'recal..."


In [24]:
resultados['melhores_parametros'].iloc[0]

"{'selection__k': 800, 'estimator__lambda': 0, 'estimator__gamma': 0, 'estimator__colsample_bytree': 1, 'estimator__alpha': 22}"

In [38]:
best_params = {'selection__k': 800,
               'estimator__lambda': 0,
               'estimator__gamma': 0,
               'estimator__colsample_bytree': 1,
               'estimator__alpha': 22}

In [41]:
pipeline = Pipeline([
                        ('scaling', MaxAbsScaler()), 
                        ('selection', SelectKBest()),
                        ('ros', RandomOverSampler(random_state=42)),
                        ('estimator', XGBClassifier(seed=42, tree_method='gpu_hist', gpu_id=0))
                        ])
    
best_xgb = pipeline.set_params(**best_params)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomOverSampler(random_state=42)' (type <class 'imblearn.over_sampling._random_over_sampler.RandomOverSampler'>) doesn't

In [42]:
best_xgb.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('scaling', MaxAbsScaler()), ('selection', SelectKBest(k=800)),
                ('ros', RandomOverSampler(random_state=42)),
                ('estimator',
                 XGBClassifier(alpha=22, base_score=0.5, booster='gbtree',
                               callbacks=None, colsample_bylevel=1,
                               colsample_bynode=1, colsample_bytree=1,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, gpu_id=0,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='', lambda=0,
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_threshold=64, max_cat_to_onehot=4,
                               max_delta_step=0, max_depth=6, max_leaves=0,
      

In [44]:
best_xgb.fit_transform(X_train_strat_vies, y_train_strat_vies)
#, X_test_strat_vies, , y_test_strat_vies

D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
D:\Downloads2\Anaconda\download\lib\site-packages\xgboost\data.py:427: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


AttributeError: 'XGBClassifier' object has no attribute 'transform'

# Predição Final

In [ ]:
best_params = {'selection__k': 800,
               'estimator__lambda': 0,
               'estimator__gamma': 0,
               'estimator__colsample_bytree': 1,
               'estimator__alpha': 22}